# Domain diagrams

This notebook creates the domain diagram figures.

# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skfmm

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
rc('text', usetex=True)
plt.rcParams['text.latex.preamble'] = [r'\usepackage{bm}']

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as cm
cmap = cm.get_cmap('Greys')
color = cmap(.5)

cdict_phi_grey = { 
    'red':  ((0.0, 1., 1.), (0.5, .7,.7), (1.0, color[0], color[0])),
    'green':((0.0, 1., 1.), (0.5, .7,.7), (1.0, color[0], color[0])),
    'blue': ((0.0, 1., 1.), (0.5, .7,.7), (1.0, color[0], color[0])),}
#     'alpha':((0.0, 1., 0.0), (0.1, 0.0, 0.0), (1.0, 1., 1.))  }
cmap_phi = LinearSegmentedColormap('cmap_phi', cdict_phi_grey)
plt.register_cmap(cmap=cmap_phi)

# Asymptotic diagrams

In [ ]:
x = np.linspace(0,2,500,endpoint=False)
z = np.linspace(0,2,500,endpoint=False)
xx, zz = np.meshgrid(x,z,indexing='ij')
h = (1 + np.sin(-np.pi*x)/3)
d0 = h[:,None]-zz
d = skfmm.distance(d0,dx=float(x[1]-x[0]),periodic=(True,False))
f = 0.5*(1 - np.tanh(d/0.1))

xs = np.linspace(0,2,20,endpoint=True)
zs = (1 + np.sin(-np.pi*xs)/3)
dxs = (xs[1]-xs[0])*np.ones_like(xs)
dzs = dxs*np.pi*np.cos(np.pi*xs)/3
nabs = np.sqrt(dxs**2 + dzs**2)
nxs =-dzs/nabs
nzs = dxs/nabs
eps = .1
xps = xs + eps*nxs
zps = zs + eps*nzs
xms = xs - eps*nxs
zms = zs - eps*nzs

fig, ax = plt.subplots(1,2,figsize=(6,3))
ax[0].fill_between(x,h.flatten(),2,color=color)
ax[0].plot(x,h.flatten(),'k',linewidth=1)
ax[1].pcolormesh(xx,zz,f,cmap='cmap_phi',vmin=0,vmax=1,edgecolors='face')
ax[1].contour(xx,zz,d,[0.],colors='k',linewidths=1,linestyles='--')

for i, axi in enumerate(ax):
    ax[i].set(ylim=[0,2],xlim=[0,2],aspect=1,xticks=[],yticks=[])
ax[0].text(.2,1.7,r'Solid $\Omega^-$',fontsize=15,horizontalalignment='left',verticalalignment='center')
ax[0].text(.2,1.1,r'Interface',fontsize=15,horizontalalignment='left',verticalalignment='center')
ax[0].text(.4,.85,r'$\partial \Omega$',fontsize=15,horizontalalignment='left',verticalalignment='center')
ax[0].text(.2,.3,r'Fluid $\Omega^+$',fontsize=15,horizontalalignment='left',verticalalignment='center')
ax[0].text(-.07,2.,'$(a)$',fontsize=15,horizontalalignment='right',verticalalignment='top')
ax[1].text(.15,1.75,r'$\phi \approx 1$',fontsize=15,horizontalalignment='left',verticalalignment='center')
ax[1].text(.15,1.05,r'$\phi = 1/2$',fontsize=15,horizontalalignment='left',verticalalignment='center')
ax[1].text(.15,.25,r'$\phi \approx 0$',fontsize=15,horizontalalignment='left',verticalalignment='center')
ax[1].text(-.07,2.,'$(b)$',fontsize=15,horizontalalignment='right',verticalalignment='top')
plt.savefig('sharp-vs-phase-diagram.png',dpi=500,bbox_inches='tight')

# Signed distance diagram

Maths for the signed distance diagram.

The interface is parameterised by $x$

$$ \boldsymbol{x} = (x,z) = (x,h(x)) = \left(x,1 - \frac{\sin(\pi x)}{3}\right)$$

The derivatives of each coordinate are therefore

$$ \boldsymbol{x}' = (1,z') = \left(1, -\frac{\pi}{3}\cos(\pi x) \right)$$

and these are therefore tangent vectors to the interface.

The second derivatives are involved in the curvature

$$ \boldsymbol{x}'' = (0, z'') = \left(0, \frac{\pi^2}{3} \sin(\pi x) \right)$$

The scale factors for the tangent vectors are their magnitude

$$ |\boldsymbol{x}'|^2 = \sqrt{1 + \frac{\pi^2}{3^2}\cos^2(\pi x)}$$

The full curvature is given by

$$ \kappa = \frac{x' z'' - z' x'' }{|\boldsymbol{x}'|^3}$$

The inverse of this quantity is the radius of curvature

In [ ]:
x = np.linspace(0,2,500,endpoint=False)
z = np.linspace(0,2,500,endpoint=False)
xx, zz = np.meshgrid(x,z,indexing='ij')
h = (1 - np.sin(np.pi*x)/3)
d0 = h[:,None]-zz
d = skfmm.distance(d0,dx=float(x[1]-x[0]),periodic=(True,False))
f = 0.5*(1 - np.tanh(d/0.1))

x0 = np.linspace(-.5,2.5,101,endpoint=True)
z0 = (1 - np.sin(np.pi*x0)/3)
dx0 = (x0[1]-x0[0])*np.ones_like(x0)
dz0 = -dx0*np.pi*np.cos(np.pi*x0)/3
nabs = np.sqrt(dx0**2 + dz0**2)
nx =-dz0/nabs
nz = dx0/nabs
ddx0 = np.zeros_like(x0)
ddz0 = dx0**2 * (-np.pi**2/3)*np.sin(np.pi*x0)
κ = (dx0*ddz0 - dz0*ddx0)/nabs**3
rc = 1/κ
rcrit = 1/κ.max()
eps = rcrit/3
# Level sets of distance function
xs = np.array([x0 + i*eps*nx for i in range(-3,4)])
zs = np.array([z0 + i*eps*nz for i in range(-3,4)])

fig, ax = plt.subplots(1,2,figsize=(6,3))
ax[0].pcolormesh(xx,zz,f,cmap='cmap_phi',vmin=0,vmax=1,edgecolors='face')
for i in range(2,5): ax[0].plot(xs[i],zs[i],color='k',linewidth=.5)
for j in range(0,len(x0),3): ax[0].plot([xs[2,j],xs[4,j]],[zs[2,j],zs[4,j]],'k',linewidth=.5)

for j in range(0,len(x0),3): ax[1].plot([xs[0,j],xs[6,j]],[zs[0,j],zs[6,j]],'gray',linewidth=.5)
for i in [3]:       ax[1].plot(xs[i],zs[i],'C0',linewidth=1)
for i in [1,2,4,5]: ax[1].plot(xs[i],zs[i],color='gray',linewidth=.5)
for i in [0,6]:     ax[1].plot(xs[i],zs[i],color='C3',linewidth=.5)
    
for i, axi in enumerate(ax):
    ax[i].set(ylim=[0,2],xlim=[0,2],aspect=1,xticks=[],yticks=[])
#     for spine in ax[i].spines:ax[i].spines[spine].set_visible(False)
j = 54
ax[0].plot([xs[2,j], xs[4,j]],
           [zs[2,j], zs[4,j]],'k',linewidth=2)
ax[0].text(.2,1.75,r'Solid $\Omega^-$',fontsize=15,horizontalalignment='left',verticalalignment='center')
ax[0].text(.15,1.2,r'Boundary',fontsize=15,horizontalalignment='left',verticalalignment='center')
ax[0].text(.5,.95,r'$\Delta \Omega$',fontsize=15,horizontalalignment='center',verticalalignment='center')
ax[0].text(.2,.25,r'Fluid $\Omega^+$',fontsize=15,horizontalalignment='left',verticalalignment='center')
ax[0].text(-0.07,2,'$(a)$',fontsize=15,horizontalalignment='right',verticalalignment='top')
ax[0].text(1.5,.25,r'$\phi \approx 0$',fontsize=15,horizontalalignment='center',verticalalignment='center')
ax[0].text(1.5,1.75,r'$\phi \approx 1$',fontsize=15,horizontalalignment='center',verticalalignment='center')
ax[0].text(1.5,1.,r'$\mathcal{O}(\varepsilon)$',fontsize=15,horizontalalignment='center',verticalalignment='center')

j = 54
width = 0.005
ax[1].scatter(xs[3,j],zs[3,j],s=5,color='k',zorder=10)
ax[1].text(xs[3,j],zs[3,j],r'$\bm{p}(s)$',horizontalalignment='left',verticalalignment='top',fontsize=15)

ax[1].scatter(xs[3,j]+.9*nx[j],zs[3,j]+.9*nz[j],s=5,color='k',zorder=10)
ax[1].text(xs[3,j]+.9*nx[j]-0.1,zs[3,j]+.9*nz[j],r'$\bm{x}$',horizontalalignment='right',verticalalignment='center',fontsize=15)

ax[1].arrow(xs[3,j],zs[3,j],.4*nx[j],.4*nz[j],zorder=3,width=width,head_width=5*width,head_length=7*width)
ax[1].text(xs[3,j]+.4*nx[j]-.1,zs[3,j]+.4*nz[j],r'$\bm{\widehat{n}}(s)$',horizontalalignment='right',verticalalignment='center',fontsize=15)

ax[1].plot([xs[3,j]+.03, xs[3,j]+.03+.9*nx[j]],
           [zs[3,j]+.02, zs[3,j]+.02+.9*nz[j]],'k--',linewidth=.75)
ax[1].text(xs[3,j]+.6*nx[j]+.04,
           zs[3,j]+.6*nz[j]+.03,r'${\sigma}$',horizontalalignment='left',verticalalignment='bottom',fontsize=15)

ax[1].text(-0.07,2,'$(b)$',fontsize=15,horizontalalignment='right',verticalalignment='top')
plt.savefig('phase-field-regions-sdf-diagram.png',dpi=500,bbox_inches='tight')